<a href="https://colab.research.google.com/github/siveshb1/Deep-Learning-Projects/blob/main/Deep%20Learning%20Basics/04_transfer_learning_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
# Get data (10% of 10 food classes from Food101) - https://www.kaggle.com/dansbecker/food-101
import zipfile

# Download the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()


--2024-04-04 07:02:26--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.145.207, 74.125.128.207, 173.194.69.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.145.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  41.9MB/s    in 4.5s    

2024-04-04 07:02:30 (36.0 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [ ]:
# How many images in each folder?
import os

# Walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = (224,224)
BATCH_SIZE = 32

train_dir = '/content/10_food_classes_10_percent/train'
test_dir = '/content/10_food_classes_10_percent/test'

train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size=IMAGE_SIZE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode='categorical')

test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SIZE,
                                             batch_size=BATCH_SIZE,
                                             class_mode='categorical')

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [ ]:
import datetime
import os

def create_tensorboard_callbacks(dir_name,experiment_name):
  current_time = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
  log_dir = os.path.join(dir_name,f'{experiment_name}_{current_time}')
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving TensorBoard log files to : {log_dir}")
  return tensorboard_callback

In [ ]:
# Let's compare the following two models
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
IMAGE_SIZE + (3,)

(224, 224, 3)

In [ ]:
def create_model(model_url,num_classes=10):
  model = tf.keras.Sequential([
    hub.KerasLayer(model_url,
                   trainable=False,
                   input_shape=IMAGE_SIZE+(3,)),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
  return model

In [ ]:
train_data_10_percent.num_classes

10

In [ ]:
resnet_model = create_model(resnet_url,train_data_10_percent.num_classes)

In [ ]:
resnet_model.compile(loss='categorical_crossentropy',
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=['accuracy'])

In [ ]:
resnet_history = resnet_model.fit(train_data_10_percent,
                                  epochs=5,
                                  steps_per_epoch=len(train_data_10_percent),
                                  validation_data=test_data,
                                  validation_steps=len(test_data),
                                  callbacks=[create_tensorboard_callbacks(dir_name='tensorflow_hub',
                                                                          experiment_name='resnet50V2')])

Saving TensorBoard log files to : tensorflow_hub/resnet50V2_2024-04-04_07-02-39
Epoch 1/5
24/24 [==============================] - 23s 654ms/step - loss: 1.9752 - accuracy: 0.3520 - val_loss: 1.2366 - val_accuracy: 0.6112
Epoch 2/5
24/24 [==============================] - 11s 463ms/step - loss: 0.9335 - accuracy: 0.7293 - val_loss: 0.8603 - val_accuracy: 0.7224
Epoch 3/5
24/24 [==============================] - 13s 546ms/step - loss: 0.6557 - accuracy: 0.8000 - val_loss: 0.7543 - val_accuracy: 0.7556
Epoch 4/5
24/24 [==============================] - 11s 477ms/step - loss: 0.4974 - accuracy: 0.8733 - val_loss: 0.7026 - val_accuracy: 0.7708
Epoch 5/5
24/24 [==============================] - 11s 451ms/step - loss: 0.4037 - accuracy: 0.9040 - val_loss: 0.6888 - val_accuracy: 0.7720
